In [2]:
!pip install paddleocr

  Using cached paddleocr-2.7.3-py3-none-any.whl (780 kB)
  Using cached premailer-3.10.0-py2.py3-none-any.whl (19 kB)
  Using cached attrdict-2.0.1-py2.py3-none-any.whl (9.9 kB)
  Using cached pdf2docx-0.5.8-py3-none-any.whl (132 kB)
  Using cached fire-0.6.0-py2.py3-none-any.whl
  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-win_amd64.whl (42.5 MB)
  Using cached imgaug-0.4.0-py2.py3-none-any.whl (948 kB)
  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl (38.5 MB)
  Using cached cssutils-2.10.2-py3-none-any.whl (398 kB)


In [3]:
import paddleocr

In [12]:
import os
from paddleocr import PaddleOCR
from PIL import Image, ImageDraw, ImageFont
import json
from uuid import uuid4
import numpy as np



ocr = PaddleOCR(use_angle_cls=False, 
                lang='en',
                  rec=False,
                )


images_folder_path  = "C:/Users/SNEH/Documents/OCR/collected_images/New_data" 


def create_image_url(filename):

    return f'http://localhost:8080/{filename}'



def convert_bounding_box(bounding_box):
    """Converts a bounding box of [x1, y1, x2, y2] into [x, y, height, width].

    Args:
    bounding_box: A list of four numbers, representing the x1, y1, x2, and y2
        coordinates of the bounding box.

    Returns:
    A list of four numbers, representing the x, y, height, and width of the
        bounding box.
    """

    x1, y1, x2, y2 = bounding_box
    x = min(x1, x2)
    y = min(y1, y2)
    width = x2 - x1
    height = y2 - y1

    return [x, y, width, height]



def extracted_tables_to_label_studio_json_file_with_paddleOCR(images_folder_path):
    label_studio_task_list = []
    for images in os.listdir(images_folder_path):
        if images.endswith('.jpg'):
            output_json = {}
            annotation_result = []

            print(images)

            output_json['data'] =  {"ocr":create_image_url(images)}

                    
            img = Image.open(f'C:/Users/SNEH/Documents/OCR/collected_images/New_data/{images}')

            img = np.asarray(img)
            image_height, image_width = img.shape[:2]


            result = ocr.ocr(img,cls=False)

            #print(result)

            for output in result:

                for item in output: 
                    co_ord = item[0]
                    text = item[1][0]

                    four_co_ord = [co_ord[0][0],co_ord[1][1],co_ord[2][0]-co_ord[0][0],co_ord[2][1]-co_ord[1][1]]

                    #print(four_co_ord)
                    #print(text)

                    bbox = {
                    'x': 100 * four_co_ord[0] / image_width,
                    'y': 100 * four_co_ord[1] / image_height,
                    'width': 100 * four_co_ord[2] / image_width,
                    'height': 100 * four_co_ord[3] / image_height,
                    'rotation': 0
                            }
                    

                    if not text:
                        continue
                    region_id = str(uuid4())[:10]
                    score = 0.5
                    bbox_result = {
                        'id': region_id, 'from_name': 'bbox', 'to_name': 'image', 'type': 'rectangle',
                        'value': bbox}
                    transcription_result = {
                        'id': region_id, 'from_name': 'transcription', 'to_name': 'image', 'type': 'textarea',
                        'value': dict(text=[text], **bbox), 'score': score}
                    annotation_result.extend([bbox_result, transcription_result])
                    #print('annotation_result :\n',annotation_result)
            output_json['predictions'] = [ {"result": annotation_result,  "score":0.97}]

            label_studio_task_list.append(output_json)
        

   
    with open('TC_label-studio_input_file.json', 'w') as f:
        json.dump(label_studio_task_list, f, indent=4)


extracted_tables_to_label_studio_json_file_with_paddleOCR(images_folder_path)

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to C:\Users\SNEH/.paddleocr/whl\det\en\en_PP-OCRv3_det_infer\en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:31<00:00, 126kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to C:\Users\SNEH/.paddleocr/whl\rec\en\en_PP-OCRv4_rec_infer\en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [01:02<00:00, 163kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to C:\Users\SNEH/.paddleocr/whl\cls\ch_ppocr_mobile_v2.0_cls_infer\ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:13<00:00, 159kiB/s] 

[2024/04/12 15:01:06] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\SNEH/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\SNEH/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_lengt

IMG20240411221903.jpg
[2024/04/12 15:01:08] ppocr DEBUG: dt_boxes num : 21, elapsed : 0.4691963195800781
[2024/04/12 15:01:09] ppocr DEBUG: rec_res num  : 21, elapsed : 1.520782232284546
IMG20240412132810.jpg
[2024/04/12 15:01:09] ppocr DEBUG: dt_boxes num : 18, elapsed : 0.08278179168701172
[2024/04/12 15:01:11] ppocr DEBUG: rec_res num  : 18, elapsed : 1.0869193077087402
IMG20240412132830.jpg
[2024/04/12 15:01:11] ppocr DEBUG: dt_boxes num : 12, elapsed : 0.05954885482788086
[2024/04/12 15:01:11] ppocr DEBUG: rec_res num  : 12, elapsed : 0.700059175491333
IMG20240412132912.jpg
[2024/04/12 15:01:12] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.4177396297454834
[2024/04/12 15:01:13] ppocr DEBUG: rec_res num  : 20, elapsed : 1.275831699371338
IMG20240412132929.jpg
[2024/04/12 15:01:13] ppocr DEBUG: dt_boxes num : 14, elapsed : 0.15022993087768555
[2024/04/12 15:01:14] ppocr DEBUG: rec_res num  : 14, elapsed : 0.6059603691101074
IMG20240412133007.jpg
[2024/04/12 15:01:14] ppocr DEBUG: dt_